https://www.imagetracking.org.uk/2018/03/piping-numpy-arrays-to-other-processes-in-python/

In [1]:
import os

In [2]:
import pickle

In [3]:
import subprocess

In [4]:
from numpy.random import rand

In [5]:
engine_str = """\
import pickle
import sys


#define some control bytes
control_data=bytes([1])
control_stop=bytes([0])


def send_data(arr,buff):
    dataStr=pickle.dumps(arr)  #pickle the data array into a byte array
    dlen=len(dataStr).to_bytes(8, byteorder='big') #find the length of the array and
    buff.write(control_data)
    buff.write(dlen)
    buff.write(dataStr)
    buff.flush() #not sure this needed

def recv_data(buff):     
    data=buff.read(1)   #read the control byte
    if data==control_data:
        data=buff.read(8)  #read the data length
        dlen=int.from_bytes(data, byteorder='big')       
        data=buff.read(dlen) #read the data 
        return pickle.loads(data) #unpickle
    elif data==control_stop:
        return None
    else:
        print('Oh no')


if __name__ == "__main__":
    while True:
        data = recv_data(buff=sys.stdin.buffer)
        if data is None:
            break
        else:
            send_data(arr=data,buff=sys.stdout.buffer)
"""

In [6]:
#define some control bytes
control_data=bytes([1])
control_stop=bytes([0])

In [7]:
def send_data(arr,buff):
    dataStr=pickle.dumps(arr)  #pickle the data array into a byte array
    dlen=len(dataStr).to_bytes(8, byteorder='big') #find the length of the array and
    buff.write(control_data)
    buff.write(dlen)
    buff.write(dataStr)
    buff.flush() #not sure this needed
     
def send_stop(buff):
    buff.write(control_stop)
    buff.flush()
    
def recv_data(buff):
    data=buff.read(1)   #read the control byte
    if data==control_data:
        data=buff.read(8)  #read the data length
        dlen=int.from_bytes(data, byteorder='big')       
        data=buff.read(dlen) #read the data        
        return pickle.loads(data)  #unpickle
    elif data==control_stop:
        return None
    else:
        print('Oh no')

In [8]:
if not os.path.exists('engine.py'):
    with open('engine.py', 'w') as f:
        f.writelines(engine_str)

In [9]:
proc = subprocess.Popen(
    'python engine.py', 
    shell=True, 
    stdin=subprocess.PIPE,                      
    stdout=subprocess.PIPE,
)

In [10]:
rand_nrs = rand(10 ** 7).tolist()

In [11]:
send_data(arr={'a': 1, 'b': rand_nrs, 'c': 3}, buff=proc.stdin)

In [12]:
recv_data(buff=proc.stdout)

{'a': 1,
 'b': [0.8444901362717046,
  0.4723855342896841,
  0.16930385023318506,
  0.37387345129484983,
  0.30096290701824846,
  0.18206396275029912,
  0.943373474118312,
  0.26548763393729913,
  0.49535856519224686,
  0.002083686655291017,
  0.41330766894553106,
  0.7086432538098094,
  0.9890660879895756,
  0.29277800582994506,
  0.7772993621446581,
  0.09635291989595507,
  0.3113284760155036,
  0.6790875891918591,
  0.6463411790348162,
  0.5885983718454035,
  0.863092249355048,
  0.9179129733541338,
  0.4577785173796559,
  0.5614580787091455,
  0.2533146923810259,
  0.775177466783375,
  0.5796318424549295,
  0.6806450121981094,
  0.0027441338603798604,
  0.8231539989998864,
  0.5515012189321109,
  0.6319325242839942,
  0.8490514220746352,
  0.925110770044491,
  0.618774601345899,
  0.20711037478100258,
  0.18519384434655572,
  0.629557765848688,
  0.2656880179721889,
  0.7930249521234828,
  0.6442045354274664,
  0.31725158433414125,
  0.7813110153961073,
  0.6859124241176425,
  0.649

In [13]:
send_stop(buff=proc.stdin)